In [39]:
# Necessary Libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
# For k-means clustering
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [5]:
# Read dataset
df=pd.read_csv("data.csv")

In [6]:
# Display Dataset
df.head()

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,0,alfa-romeo,Alfa Romeo GTV,red,10/1995,1995,1300,148,201,Manual,Petrol,"10,9 l/100 km",260 g/km,160500.0,2.0 V6 TB
1,1,alfa-romeo,Alfa Romeo 164,black,02/1995,1995,24900,191,260,Manual,Petrol,NaN,- (g/km),190000.0,"Q4 Allrad, 3.2L GTA"
2,2,alfa-romeo,Alfa Romeo Spider,black,02/1995,1995,5900,110,150,Unknown,Petrol,NaN,- (g/km),129000.0,ALFA ROME 916
3,3,alfa-romeo,Alfa Romeo Spider,black,07/1995,1995,4900,110,150,Manual,Petrol,"9,5 l/100 km",225 g/km,189500.0,2.0 16V Twin Spark L
4,4,alfa-romeo,Alfa Romeo 164,red,11/1996,1996,17950,132,179,Manual,Petrol,"7,2 l/100 km",- (g/km),96127.0,"3.0i Super V6, absoluter Topzustand !"


In [7]:
# Display information about dataset
# Begin of data preprocessing
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251079 entries, 0 to 251078
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                251079 non-null  int64  
 1   brand                     251079 non-null  object 
 2   model                     251079 non-null  object 
 3   color                     250913 non-null  object 
 4   registration_date         251075 non-null  object 
 5   year                      251079 non-null  object 
 6   price_in_euro             251079 non-null  object 
 7   power_kw                  250945 non-null  object 
 8   power_ps                  250950 non-null  object 
 9   transmission_type         251079 non-null  object 
 10  fuel_type                 251079 non-null  object 
 11  fuel_consumption_l_100km  224206 non-null  object 
 12  fuel_consumption_g_km     251079 non-null  object 
 13  mileage_in_km             250927 non-null  f

In [8]:
# Drop columns that will not contribute to the model.
# Drop column registration_date because it looks a lot like year. Same year only as an extra there is a month.
df.drop(columns=['Unnamed: 0', 'offer_description', 'registration_date'], inplace=True)

# Display total null value number column by column
df.isnull().sum()

brand                           0
model                           0
color                         166
year                            0
price_in_euro                   0
power_kw                      134
power_ps                      129
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    26873
fuel_consumption_g_km           0
mileage_in_km                 152
dtype: int64

In [9]:
# Fill in empty categorical data (color) with unknown
df['color'] = df['color'].fillna('unknown')

# Display total null value number column by column
df.isnull().sum()

brand                           0
model                           0
color                           0
year                            0
price_in_euro                   0
power_kw                      134
power_ps                      129
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    26873
fuel_consumption_g_km           0
mileage_in_km                 152
dtype: int64

In [10]:
# Converting numeric objects to numeric variables
df['fuel_consumption_l_100km'] = df['fuel_consumption_l_100km'].str.replace(' l/100 km', '', regex=False)
df['fuel_consumption_l_100km'] = df['fuel_consumption_l_100km'].str.replace(',', '.', regex=False)
df['fuel_consumption_l_100km'] = pd.to_numeric(df['fuel_consumption_l_100km'], errors='coerce')

df['fuel_consumption_g_km'] = df['fuel_consumption_g_km'].replace(r'-.*', pd.NA, regex=True)
df['fuel_consumption_g_km'] = df['fuel_consumption_g_km'].str.replace(' g/km', '', regex=False)
df['fuel_consumption_g_km'] = pd.to_numeric(df['fuel_consumption_g_km'], errors='coerce')

df['price_in_euro'] = df['price_in_euro'].str.replace(' €', '', regex=False)
df['price_in_euro'] = df['price_in_euro'].str.replace('.', '', regex=False)
df['price_in_euro'] = pd.to_numeric(df['price_in_euro'], errors='coerce')

df['power_kw'] = pd.to_numeric(df['power_kw'], errors='coerce')

df['power_ps'] = pd.to_numeric(df['power_ps'], errors='coerce')

# Display total null value number column by column
df.isnull().sum()

brand                           0
model                           0
color                           0
year                            0
price_in_euro                 199
power_kw                      293
power_ps                      210
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    29172
fuel_consumption_g_km       42250
mileage_in_km                 152
dtype: int64

In [11]:
# Fill missing numeric values with the mean
df['fuel_consumption_l_100km'] = df['fuel_consumption_l_100km'].fillna(df['fuel_consumption_l_100km'].mean())
df['fuel_consumption_g_km'] = df['fuel_consumption_g_km'].fillna(df['fuel_consumption_g_km'].mean())
df['price_in_euro'] = df['price_in_euro'].fillna(df['price_in_euro'].mean())
df['power_kw'] = df['power_kw'].fillna(df['power_kw'].mean())
df['power_ps'] = df['power_ps'].fillna(df['power_ps'].mean())
df['mileage_in_km'] = df['mileage_in_km'].fillna(df['mileage_in_km'].mean())

# Display total null value number column by column
df.isnull().sum()

brand                       0
model                       0
color                       0
year                        0
price_in_euro               0
power_kw                    0
power_ps                    0
transmission_type           0
fuel_type                   0
fuel_consumption_l_100km    0
fuel_consumption_g_km       0
mileage_in_km               0
dtype: int64

In [12]:
# Display information about dataset
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251079 entries, 0 to 251078
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   brand                     251079 non-null  object 
 1   model                     251079 non-null  object 
 2   color                     251079 non-null  object 
 3   year                      251079 non-null  object 
 4   price_in_euro             251079 non-null  float64
 5   power_kw                  251079 non-null  float64
 6   power_ps                  251079 non-null  float64
 7   transmission_type         251079 non-null  object 
 8   fuel_type                 251079 non-null  object 
 9   fuel_consumption_l_100km  251079 non-null  float64
 10  fuel_consumption_g_km     251079 non-null  float64
 11  mileage_in_km             251079 non-null  float64
dtypes: float64(6), object(6)
memory usage: 23.0+ MB


,brand,model,color,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km
0,alfa-romeo,Alfa Romeo GTV,red,1995,1300.0,148.0,201.0,Manual,Petrol,10.900000,260.000000,160500.0
1,alfa-romeo,Alfa Romeo 164,black,1995,24900.0,191.0,260.0,Manual,Petrol,6.487279,140.802646,190000.0
2,alfa-romeo,Alfa Romeo Spider,black,1995,5900.0,110.0,150.0,Unknown,Petrol,6.487279,140.802646,129000.0
3,alfa-romeo,Alfa Romeo Spider,black,1995,4900.0,110.0,150.0,Manual,Petrol,9.500000,225.000000,189500.0
4,alfa-romeo,Alfa Romeo 164,red,1996,17950.0,132.0,179.0,Manual,Petrol,7.200000,140.802646,96127.0


In [24]:
# En çok geçen 5 rengi bul
top5_colors = df['color'].value_counts().nlargest(5).index

# Diğer tüm renkleri 'others' olarak değiştir
df['color'] = df['color'].apply(lambda x: x if x in top5_colors else 'others')


In [25]:
# En çok geçen 5 brand bul
top5_brands = df['brand'].value_counts().nlargest(5).index

# Diğer tüm renkleri 'others' olarak değiştir
df['brand'] = df['brand'].apply(lambda x: x if x in top5_brands else 'others')


In [27]:
# En çok geçen 5 model bul
top5_models = df['model'].value_counts().nlargest(5).index

# Diğer tüm renkleri 'others' olarak değiştir
df['model'] = df['model'].apply(lambda x: x if x in top5_models else 'others')

In [31]:
# Sayısal olmayanları otomatik olarak NaN yap
df['year'] = pd.to_numeric(df['year'], errors='coerce')

df['year'] = df['year'].fillna(df['year'].median())

# Şimdi float oldu, ama geçersizler NaN
print(df['year'].dtype)  # float64


float64


In [33]:
# En çok geçen 5 fuel bul
top5_fuels = df['fuel_type'].value_counts().nlargest(5).index

# Diğer tüm renkleri 'others' olarak değiştir
df['fuel_type'] = df['fuel_type'].apply(lambda x: x if x in top5_fuels else 'others')

In [36]:
print(f"Unique values")
for column in df.columns:
    print(f"{column}: {df[column].nunique()}")

Unique values
brand: 6
model: 6
color: 6
year: 33
price_in_euro: 18081
power_kw: 512
power_ps: 521
transmission_type: 4
fuel_type: 6
fuel_consumption_l_100km: 268
fuel_consumption_g_km: 492
mileage_in_km: 71767


In [ ]:
#Encoding
le = LabelEncoder()
df['color_encoded'] = le.fit_transform(df['color'])
df['brand_encoded'] = le.fit_transform(df['brand'])
df['model_encoded'] = le.fit_transform(df['model'])
df['fuel_type_encoded'] = le.fit_transform(df['fuel_type'])
df['transmission_type_encoded'] = le.fit_transform(df['transmission_type'])

df = df.drop('color', axis=1)
df = df.drop('brand', axis=1)
df = df.drop('model', axis=1)
df = df.drop('fuel_type', axis=1)
df = df.drop('transmission_type', axis=1)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251079 entries, 0 to 251078
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year                       251079 non-null  float64
 1   price_in_euro              251079 non-null  float64
 2   power_kw                   251079 non-null  float64
 3   power_ps                   251079 non-null  float64
 4   fuel_consumption_l_100km   251079 non-null  float64
 5   fuel_consumption_g_km      251079 non-null  float64
 6   mileage_in_km              251079 non-null  float64
 7   color_encoded              251079 non-null  int64  
 8   brand_encoded              251079 non-null  int64  
 9   model_encoded              251079 non-null  int64  
 10  fuel_type_encoded          251079 non-null  int64  
 11  transmission_type_encoded  251079 non-null  int64  
dtypes: float64(7), int64(5)
memory usage: 23.0 MB
